In [1]:
%pip install tf-keras
%pip install transformers

In [6]:
import numpy as np
import pandas as pd
import preprocess
from transformers import pipeline

In [6]:
inputData = pd.read_csv("data/analyst_ratings_processed.csv")

inputData.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [9]:
preprocessedData = preprocess.Preprocess_Tweets(inputData)

preprocessedData.head()

d:\ASU\DM\HW2\.venv\Lib\site-packages\pandas\core\strings\object_array.py:172: FutureWarning: Possible nested set at position 1
  pat = re.compile(pat, flags=flags)


,Unnamed: 0,title,date,stock,Text_Cleaned
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,stocks that hit 52 week highs on friday
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,stocks that hit 52 week highs on wednesday
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,71 biggest movers from friday
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,46 stocks moving in fridays mid day session
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,b of a securities maintains neutral on agilent...


In [32]:
preprocessedData.to_csv("data/PreprocessedData.csv")

In [7]:
preprocessedData = pd.read_csv("data/PreprocessedData.csv")

In [1]:
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

d:\ASU\DM\HW2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
stocksToProcess = ["JNJ", "NFLX", "KO", "PFE", "PG", "MA", "HD", "NVDA", "GOOG", "TSLA"]

In [25]:
preprocessedData.head()

,Unnamed: 0.1,Unnamed: 0,title,date,stock,Text_Cleaned,Score
0,0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,stocks that hit 52 week highs on friday,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,stocks that hit 52 week highs on wednesday,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,71 biggest movers from friday,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,46 stocks moving in fridays mid day session,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,b of a securities maintains neutral on agilent...,"{'neg': 0.0, 'neu': 0.864, 'pos': 0.136, 'comp..."


In [29]:
for stock in stocksToProcess[1:]:
    # Perform whatever processing or analysis you need on x using pipe(x)
    df = preprocessedData[preprocessedData['stock'] == stock].copy()
    label_ = []
    score_ = []
    for x in df.index:
        results = pipe(df['Text_Cleaned'][x])
        label_.append(results[0]["label"])
        score_.append(results[0]["score"])

    df['label'] = np.array(label_)
    df['score'] = np.array(score_)
    nameCsv = "data/data_" + stock + ".csv"
    df.to_csv(nameCsv)


In [69]:
for stock in stocksToProcess:
    nameCsv = "data/data_" + stock + ".csv"
    # Read the data into a DataFrame
    data = pd.read_csv(nameCsv)

    # Convert the "Date" column to datetime
    data['date'] = data['date'].str.split().str[0]
    data['date'] = pd.to_datetime(data['date'])

    def calculate_average_score(group):
        positive_score, negative_score, neutral_score = 0, 0, 0
        pos = group[group['label'] == 'positive']['score'].mean()
        neg = -group[group['label'] == 'negative']['score'].mean()
        neu = 0
        if (not np.isnan(pos)): 
            positive_score = pos
        if (not np.isnan(neg)): 
            negative_score = neg
        return positive_score + negative_score + neutral_score

    # Group by date and calculate average score
    grouped_data = data.groupby('date').apply(calculate_average_score).reset_index(name='Average_Score')
    grouped_data['Stock'] = data['stock']


    outputName = "data/Date_filtered_" + stock + ".csv"
    # Save the grouped data to CSV
    grouped_data.to_csv(outputName, index=False)

C:\Users\mithi\AppData\Local\Temp\ipykernel_11188\395515274.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby('date').apply(calculate_average_score).reset_index(name='Average_Score')


AttributeError: 'DataFrame' object has no attribute 'append'

In [55]:
df = data.groupby('date')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,date,stock,Text_Cleaned,Score,label,score
0,595269,595269,606131.0,10 Biggest Price Target Changes For Wednesday,2020-06-10,HD,10 biggest price target changes for wednesday,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,0.999812
1,595270,595270,606132.0,"UBS Maintains Buy on Home Depot, Raises Price ...",2020-06-10,HD,ubs maintains buy on home depot raises price t...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",positive,0.999566
2,595271,595271,606133.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,HD,stocks that hit 52 week highs on friday,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",positive,0.998992
